In [ ]:
'''
   Copyright 2023 Spacebel s.a.

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
'''

# OGC API Features

## Overview

This notebook explains the use of the OGC API Features [[RD20]](#RD20) interface with GeoJSON and other response formats. It uses the `OWSLib` [[RD31]](#RD31) library to access part of the interface. The visualisation of search results is borrowed from the ODC notebook available at [[RD19]](#RD19).

In [ ]:
%pip install geopandas
%pip install owslib
%pip install folium matplotlib mapclassify

In [ ]:
%pip install jsonpath_ng
# %pip install glom
%pip install rdflib
%pip install -U prettytable

In [ ]:
import folium
import folium.plugins
import geopandas as gpd
import shapely.geometry
import pandas as pd  
import json
import requests

from xml.dom import minidom
from IPython.display import HTML, display
from IPython.display import Markdown as md
from owslib.ogcapi.features import Features
from urllib.parse import urlparse, parse_qsl
from matplotlib import pyplot as plt
from PIL import Image
from io import BytesIO
from branca.element import Figure
from prettytable import PrettyTable


def convert_bounds(bbox, invert_y=False):
    """
    Helper method for changing bounding box representation to leaflet notation

    ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))``
    """
    x1, y1, x2, y2 = bbox
    if invert_y:
        y1, y2 = y2, y1
    return ((y1, x1), (y2, x2))



def curl_command( url: str, method: str = "GET" ) -> str:
    """
    Convert request URL to equivalent curl GET or POST command-line (bash).
    """
    c = "curl -X " + method
    res = urlparse(url)
    if "GET" in method:
        c = c + " -G " + res.scheme + "://" + res.netloc + res.path
    else:
        c = c + " " + res.scheme + "://" + res.netloc + res.path + " \\\n\t--data-raw '{"
    
    lst = parse_qsl(res.query)
    
    first = True 
    for i in lst:
        # print(i[0])
        # add \ to end of previous line
        if "GET" in method:
            c = c + ' \\\n\t--data-urlencode "'+i[0]+'='+i[1]+'"'
        else:
            if not(first):
                c = c + ','
            c = c + '\n\t\t"'+i[0]+'": '+i[1]
        first = False
        
    if "POST" in method:
        c = c + " \\\n\t}'"
    return c


def display_previews(results):
    """
    Helper method for displaying a grid of quicklooks (if available)
    """
    # create figure
    fig = plt.figure(figsize=(20, 20))
  
    # setting values to rows and column variables for the image grid
    rows = 8
    columns = 2
    pos = 1

    for item in results.items():
        # print(item.id)
        assets = item.assets
        try:
            # print("found thumbnail", assets['thumbnail'].href)
            url = assets['thumbnail'].href
            response = requests.get(url)
            Image1 = Image.open(BytesIO(response.content))
            # display at position 'pos'
            fig.add_subplot(rows, columns, pos)
            pos = pos+1
            # show the image
            plt.imshow(Image1)
            plt.axis('off')
            plt.title(item.id)
        except:
            pass
    return

def display_gdf_plot(results):
    """
    Helper method for displaying results as dataframe plot
    """
    # https://github.com/opendatacube/odc-stac (License Apache 2.0)
    # https://odc-stac.readthedocs.io/en/latest/notebooks/stac-load-e84-aws.html#Plot-STAC-Items-on-a-Map

    # items = list(results.get_items())
    # print(f"Found: {len(items):d} datasets")

    # Convert STAC items into a GeoJSON FeatureCollection
    if (type(results) is dict):
        stac_json = results
    else:
        stac_json = results.get_all_items_as_dict()

    gdf = gpd.GeoDataFrame.from_features(stac_json, "epsg:4326")

    fig = gdf.plot(
        "date",
        edgecolor="black",
        categorical=True,
        aspect="equal",
        alpha=0.5,
        figsize=(6, 12),
        legend=True,
        legend_kwds={"loc": "upper left", "frameon": False, "ncol": 1},
    )
    _ = fig.set_title("Query Results")

    # gdf
    return

def display_date_distribution(results):
    """
    Helper method for displaying number of results per year/month as bar chart
    """
    if (type(results) is dict):
        stac_json = results
    else:
        stac_json = results.get_all_items_as_dict()
    gdf = gpd.GeoDataFrame.from_features(stac_json)
    
    # items = list(results.get_items())
    # stac_json = results.get_all_items_as_dict()
    # gdf = gpd.GeoDataFrame.from_features(stac_json)
    
    gdf['start_datetime'] = gdf['date'].map(lambda dt: dt.split("/")[0])

    gdf['date'] = pd.to_datetime(gdf['start_datetime'])
    # create a representation of the month with strfmt
    gdf['year_month'] = gdf['date'].map(lambda dt: dt.strftime('%Y-%m'))
    grouped_df = gdf.groupby('year_month')['year_month'].size().to_frame("count").reset_index()
    grouped_df.plot(kind='bar', x='year_month', y='count')
    return

def display_value_distribution(results, column):
    """
    Helper method for displaying number values in column as bar chart
    """
    if (type(results) is dict):
        stac_json = results
    else:
        stac_json = results.get_all_items_as_dict()
    gdf = gpd.GeoDataFrame.from_features(stac_json)

    # gdf['date'] = pd.to_datetime(gdf['start_datetime'])
    # create a representation of the month with strfmt
    # gdf['year_month'] = gdf['date'].map(lambda dt: dt.strftime('%Y-%m'))

    try:
        grouped_df = gdf.groupby(column)[column].size().to_frame("count").reset_index()
        grouped_df.plot(kind='bar', x=column, y='count')
    except:
        print(column + " values are not available.")

    return

def display_map(results):
    """
    Helper method for displaying results on a map
    """
    # https://github.com/python-visualization/folium/issues/1501
    # if (is_dict):
    if (type(results) is dict):
        stac_json = results
    else:
        stac_json = results.get_all_items_as_dict()
    gdf = gpd.GeoDataFrame.from_features(stac_json, "epsg:4326")
    
    fig = Figure(width="800px", height="500px")
    map1 = folium.Map()
    fig.add_child(map1)

    # folium.GeoJson(
    #    shapely.geometry.box(*bbox),  # ??
    #    style_function=lambda x: dict(fill=False, weight=1, opacity=0.7, color="olive"),
    #    name="Query",
    # ).add_to(map1)

    gdf.explore(
        "date", # "start_datetime",
        categorical=True,
    #    tooltip=[
    #        "title", "datetime", "start_datetime", "platform", "instruments"    
    #    ],
        popup=True,
        style_kwds=dict(fillOpacity=0.1, width=2),
        name="Results",
        m=map1,
    )

    map1.fit_bounds(bounds=convert_bounds(gdf.unary_union.bounds))
    display(fig)
    return


In [ ]:
URL_LANDING_PAGE =  'https://emc.spacebel.be/'

In [ ]:
COLLECTION_ID1 = 'PROBA.CHRIS.1A'
COLLECTION_ID2 = 'SPOT-6.and.7.ESA.archive'  
COLLECTION_ID4 = 'Deimos-1.and.2.ESA.archive' 
COLLECTION_ID2_CLOUDS = 'LANDSAT.ETM.GTC'
COLLECTION_ID3_CLOUDS = 'IKONOS.ESA.archive'

In [ ]:
# Specialize the Features class
from owslib.ogcapi.features import Features
from owslib.util import Authentication
from requests import Request
from copy import deepcopy
import urllib.parse

class ApiFeatures(Features):
    
    search_kwargs = []
    search_path = ""

    def __init__(self, url: str, json_: str = None, timeout: int = 30,
                 headers: dict = None, auth: Authentication = None):
        # __doc__ = Collections.__doc__  # noqa
        super().__init__(url, json_, timeout, headers, auth)
        
    def collection_item(self, collection_id: str, identifier: str) -> dict:
        """
        implements /collection/{collectionId}/items
        @returns: feature results
        """
        
        self.search_kwargs = []
        self.search_path = f'collections/{collection_id}/items/{identifier}'
        return super().collection_item(collection_id, identifier)
        
    def collection_items(self, collection_id: str, **kwargs: dict) -> dict:
        """
        implements /collection/{collectionId}/items
        @returns: feature results
        """
        self.search_kwargs = deepcopy(kwargs)
        if 'bbox' in kwargs:
            self.search_kwargs['bbox'] = ','.join(list(map(str, kwargs['bbox'])))
        if 'datetime_' in kwargs:
            self.search_kwargs['datetime'] = kwargs['datetime_']

        # if 'cql' in kwargs:
        #    LOGGER.debug('CQL query detected')
        #    kwargs2 = deepcopy(kwargs)
        #    cql = kwargs2.pop('cql')
        #    path = f'collections/{collection_id}/items?{urlencode(kwargs2)}'
        #    return self._request(method='POST', path=path, data=cql, kwargs=kwargs2)
        # else:
        self.search_path = f'collections/{collection_id}/items'
        #    return self._request(path=path, kwargs=kwargs)
        
        return super().collection_items(collection_id, **kwargs)
    
    def url_with_parameters(self) -> str:
        """Returns previous item search url with parameters, appropriate for a GET request."""
        
        # params = self._clean_params_for_get_request()
        # print("url_with_params: ", self.path, " kwargs: " , self.search_kwargs)
        
        # params = []
        params = self.search_kwargs
        request = Request("GET", self.url + self.search_path, params=params)
        url = request.prepare().url
        if url is None:
            raise ValueError("Could not construct a full url")
        return urllib.parse.unquote(url)


In [ ]:
def create_example_table_as_html( mediatypes , chapter: int):
    # Use HTML output as the markdown library output cannot be correctly 
    # be converted to Jupyterbook output.

    text="<table><tr><th align='left'>Example</th><th align='left'>Media type</th></tr>" 

    digit1 = chapter
    digit2 = 1
    for f in mediatypes:
    
        f2 = f
        # f2 = f.replace(";", ";<br>")  # works in VS-Code, not in JupyterBook
        # f2 = f2.replace('"', '\\"')
        #       + '| ' + 'Example '+str(digit1)+ '.' + str(digit2)  \
     
        text = text \
            + '<tr>' + '<td align="left"><a href="#example_' + str(digit1) + '_' + str(digit2)+'">' \
            + 'Example '+str(digit1)+ '.' + str(digit2)+'</a>'  \
            + '</td><td align="left">' + f2  + '</td></tr>`\n'
        digit2 = digit2+1
    
    text = text + "</table>"
    return text

In [ ]:
def create_examples_as_md(mediatypes, chapter: int, resource: str, resource_type: str ):
    """
    Insert examples for the resource in all mediatypes and number the
    examples starting n.1, n.2 with n is chapter number provided.
    """

    text = ""
    digit1 = chapter
    digit2 = 1
    for f in mediatypes:
    
        f2 = urllib.parse.quote(f)
    
        if ("?" in resource):
            request = resource +'&httpAccept='+f2 
        else:
            request = resource +'?httpAccept='+f2
       
        # insert an HTML anchor for each example to jump to 
        text = text + '\n' \
            + "<a id='example_" + str(digit1) + '_' + str(digit2) + "'></a>" + '\n' 
    
        text = text + '\n' + '\n' + '\n' \
            + '**Example: '+str(digit1)+ '.' + str(digit2) +'**' + '\n' \
            + '>  Represent ' + resource_type + ' in `' + f + '` media format (`httpAccept`).\n' \
            + '> \n' 
    
        # print("DEBUG f = ", f, "request = ", request)
        # profile string at end of media type may contain "json"...
        # thus xml tested for first.
        
        # 
        curl_str = curl_command(request)
        text = text + "```shell\n" + curl_str + "\n```\n"
        
        if ("xml" in f ):
            # formatting XML output
            response = requests.get(request)
            if response.status_code==200:
                xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ', newl='')
                text = text + "```xml\n" + xmlstr + "\n```\n"
            else:
                text = text + "Media type not supported.\n"
            
        elif ("text/" in f ):
            # formatting text output
            response = requests.get(request)
            if response.status_code==200:
                # xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ', newl='')
                # text = text + response.text + "\n"
                text = text + "```\n" + response.text + "\n```\n"
            else:
                text = text + "Media type not supported.\n"
    
        elif ("json" in f ):
            # formatting JSON output
            # print("URL:",request)
            response = requests.get(request)
            if response.status_code==200:
                data = json.loads(response.text)
                jstr = json.dumps(data, indent=3)
                text = text + '\n' + "```json\n" + jstr + "\n```\n"
            else:
                text = text + "Media type not supported.\n"
        else:
            text = text + "ERROR: Response visualisation not yet supported."
           
        digit2 = digit2+1
    
    return text

In [ ]:
w = ApiFeatures(URL_LANDING_PAGE)
w.url

### Conformance

In [ ]:
md(f"The conformance classes supported by OGC API Features interface are listed at `{URL_LANDING_PAGE + 'conformance'}`.")

In [ ]:
conformance = w.conformance()
conformance

Same request with `curl`.

In [ ]:
curl_str = curl_command(URL_LANDING_PAGE + 'conformance')
md("```shell\n" + curl_str + "\n```\n")

The conformance response lists supported conformance classes.
For example:

- `http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/queryables` from OGC API Features Part 3 [[RD21]](#RD21) indicates that collection-specific search parameters are advertised at `/collections/{collection-id}/queryables`.
- `http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/queryables-query-parameters` from OGC API Features Part 3 [[RD21]](#RD21) indicates that queryables advertised at `/collections/{collection-id}/queryables` can be used as HTTP query parameter.
- `http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/filter` from OGC API Features Part 3 [[RD21]](#RD21) indicates that queryables advertised at `/collections/{collection-id}/queryables` can be used in a CQL2 filter expression.

These conformance classes have an equivalent conformance class declared by the STAC API.  The STAC conformance classes are reported inside the Landing Page.


### Access API Description

In [ ]:
URL_OAI_DEFINITION = URL_LANDING_PAGE + 'api'
# URL_OAI_DEFINITION

In [ ]:
curl_str = curl_command(URL_OAI_DEFINITION)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
# Get OpenAPI definition
jstr = json.dumps(w.api(), indent=3)
md("```json\n" + jstr + "\n```\n")

### Collections

Collections are accessible at the endpoint identified via rel="data" in the landing page.

In [ ]:
collections = w.collections()
collections

In [ ]:
print(f"{collections['numberMatched']} items found.")

In [ ]:
df = pd.json_normalize(collections, record_path=['collections'], max_level = 0)
df[['id', 'title']]
# df

In [ ]:
# Collections with itemType = 'feature'
feature_collections = w.feature_collections()
feature_collections

## Collection

In [ ]:
fc = w.collection('series')
fc['id']

In [ ]:
fc

The Python library provides access to `collection_queryables` which correspond to the queryables that can be used inside `filter` expressions.

In [ ]:
# query parameters for 'series' collection.
w.collection_queryables('series')

In [ ]:
fc = w.collection('datasets')
fc['id']

In [ ]:
services = w.collection('services')
services['id']

In [ ]:
services['title']

In [ ]:
services['description']

In [ ]:
w.collection_queryables('services')

## Item Search (Collections)


### Access API description

The following search parameters for `series` (collections) are declared in the `/collections/series/items` section of the OpenAPI definition.  The `x-value` column provides the name of the corresponding OpenSearch parameter.

In [ ]:
response = requests.get(URL_OAI_DEFINITION)
apidoc = json.loads(response.text)

ref = apidoc['paths']['/collections/series/items']['get']['parameters']
df = pd.json_normalize(ref, max_level = 0)
df[['name','description','x-value']].sort_values(by=['name'])

### Search response formats

The following response formats (media types) for `series` (collections) are declared in the `/collections/series/items` section of the OpenAPI definition.  The media type can be requested via the `Accept` header parameter or the `httpAccept` query parameter.  

In [ ]:
ref = apidoc['paths']['/collections/series/items']['get']['responses']['200']['content']
df = pd.json_normalize(ref, max_level = 0)
sorted(ref.keys())

### Search by free text

**Example: 1.1**  
>  Search collections by free text (`query`). 

In [ ]:
results = w.collection_items(
    collection_id = 'series', 
    limit = 10, 
    query = 'temperature' 
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['title','abstract']]

In [ ]:
jstr = json.dumps(results, indent=3)
md("```json\n" + jstr + "\n```\n")

### Search by title

**Example: 1.2**  
>  Search collections by title (`title`).  

In [ ]:
results = w.collection_items(
    collection_id = 'series', 
    limit = 10, 
    title = 'Total column' 
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['title']]

### Search by platform

**Example: 1.3**  
>  Search collections by platform (`platform`).  

In [ ]:
results = w.collection_items(
    collection_id = 'series', 
    limit = 50, 
    platform = 'PROBA-1' 
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['title','abstract']]

### Search by organisation

**Example: 1.3**  
>  Search collections by organisation (`organisationName`). 

In [ ]:
results = w.collection_items(
    collection_id = 'series', 
    limit = 20, 
    platform = 'PROBA-1',
    organisationName = 'ESA/ESRIN' 
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['title','abstract']]

### Search by identifier

**Example: 1.4**  
>  Search collections by identifier.

In [ ]:
# Keep for future use.
series_id = results['features'][0]['id']
series_id

In [ ]:
results = w.collection_item(
    collection_id = 'series', 
    identifier = series_id
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
results['properties']['title']

### Search by DOI

**Example: 1.5**  
>  Search collections by DOI (`doi`). 

In [ ]:
results = w.collection_items(
    collection_id = 'series', 
    limit = 20, 
    doi = '10.5270/esa-qoe849q',  # TropForest
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['title']]

### Search by concept

**Example: 1.6**  
>  Search collections by concept (`classifiedAs`).

Collection metadata includes platform, instrument and science keywords, including the URI of these concepts expressed in the ESA Thesauri and NASA GCMD thesauri. The URI of these concepts can be used as search parameter.

In [ ]:
results = w.collection_items(
    collection_id = 'series', 
    limit = 20, 
    classifiedAs = 'https://earth.esa.int/concept/b3979ff2-d27d-5f22-9e06-a18c5759d9a5',  # PROBA-1
    organisationName = 'ESA/ESRIN' 
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['title','abstract']]

### Search by keyword

**Example: 1.7**  
>  Search collections by keyword (`subject`).

In [ ]:
results = w.collection_items(
    collection_id = 'series', 
    limit = 5, 
    subject = 'ice',  
    organisationName = 'ESA/ESRIN' 
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['title','categories']]
# gdf

## Item Search (Services)

### Access API description

The following search parameters for `services` (services and applications) are declared in the `/collections/services/items` section of the OpenAPI definition.  The `x-value` column provides the name of the corresponding OpenSearch parameter.

In [ ]:
URL_OAI_DEFINITION = URL_LANDING_PAGE + 'api'
URL_OAI_DEFINITION

Request using `curl`.

In [ ]:
curl_str = curl_command(URL_OAI_DEFINITION)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(URL_OAI_DEFINITION)
apidoc = json.loads(response.text)
ref = apidoc['paths']['/collections/services/items']['get']['parameters']
df = pd.json_normalize(ref, max_level = 0)
df[['name','description','x-value']].sort_values(by=['name'])

In [ ]:
jstr = json.dumps(ref, indent=3)
md("```json\n" + jstr + "\n```\n")

### Search response formats

The following response formats (media types) for `services` (granules) are declared in the `/collections/services/items` section of the OpenAPI definition.  The media type can be requested via the `Accept` header parameter or the `httpAccept` query parameter.  

In [ ]:
ref = apidoc['paths']['/collections/services/items']['get']['responses']['200']['content']
df = pd.json_normalize(ref, max_level = 0)
sorted(ref.keys())

### Search by free text

```{index} double: searchRetrieve ; query
```

**Example: 2.1**  
>  Search services by free text (`query`). The `query` parameter is defined in the SRU specification [[RD07]](#RD07).

In [ ]:
results = w.collection_items(
    collection_id = 'services', 
    limit = 20, 
    query = 'toolbox' 
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['title','abstract']]

In [ ]:
jstr = json.dumps(results, indent=3)
md("```json\n" + jstr + "\n```\n")

### Search by title

**Example: 2.2**  
>  Search services by title (`title`).

In [ ]:
results = w.collection_items(
    collection_id = 'services', 
    limit = 20, 
    title = 'Toolbox' 
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['title','abstract']]

### Search by platform

**Example: 2.3**  
>  Search services by platform (`platform`). 

In [ ]:
results = w.collection_items(
    collection_id = 'services', 
    limit = 20, 
    platform = 'GOCE' 
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['title','abstract']]

### Search by organisation

**Example: 2.3**  
>  Search services by organisation (`organisationName`). 

In [ ]:
results = w.collection_items(
    collection_id = 'services', 
    limit = 20, 
    organisationName = 'ESA/ESRIN' 
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['title','abstract']]

### Search by identifier

**Example: 2.4**  
>  Search services by identifier. 

In [ ]:
# Keep for future use.
service_id = results['features'][0]['id']
service_id

In [ ]:
results = w.collection_item(
    collection_id = 'services', 
    identifier = service_id
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
results['properties']['title']

### Search by concept

**Example: 2.5**  
>  Search services by concept (`classifiedAs`). 

Service metadata includes platform, instrument and science keywords, including the URI of these concepts expressed in the ESA thesauri and NASA GCMD thesauri.  The URI of these concepts can be used as search parameter.  

In the current version of the software, the following concept URIs are supported:

* ESA thesaurus platform URI
* ESA thesaurus instrument URI
* ESA thesaurus earth topic URI
* ESA thesaurus tool category URI (experimental)
* GCMD platform URI
* GCMD instrument URI
* GCMD science keyword URI

In [ ]:
results = w.collection_items(
    collection_id = 'services', 
    limit = 20, 
    # GOCE toolbox, gravitational ...
    # https://earth.esa.int/concept/bd09a085-0d60-5bad-abe4-a4555e80e9b9  # Tools / Visualisation
    classifiedAs = 'https://earth.esa.int/concept/1abfac39-23bf-561f-a765-76da42a79d44',  # GOCE
    organisationName = 'ESA/ESRIN' 
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['title','abstract']]

### Search by keyword

**Example: 2.6**  
>  Search services by keyword (`subject`). 

In [ ]:
results = w.collection_items(
    collection_id = 'services', 
    limit = 5, 
    subject = 'Solid Earth',  
    organisationName = 'ESA/ESRIN' 
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['title','abstract']]

### Search by offering

**Example: 2.7**  
>  Search services by offering (`offering`). 

In [ ]:
results = w.collection_items(
    collection_id = 'services', 
    limit = 5, 
    # offering = 'http://www.opengis.net/spec/eopad-geojson/req/docker/image',
    # offering = 'image',  
    offering = 'wcs',
    # offering = 'docker',  # no results
    organisationName = 'ESA/ESRIN' 
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['title','abstract']]

## Item Search (Granules)

### Access API description

The following search parameters for `datasets` (granules) are declared in the `/collections/datasets/items` section of the OpenAPI definition.  The `x-value` column provides the name of the corresponding OpenSearch parameter.  Not all search parameters are supported for a particular EO collection (`parentIdentifier`).  The collection-specific OpenSearch Description Document (OSDD) provides the correct list. Future versions of the interface will support the `/queryables` response for each EO collections, facilitating access to the correct list.

In [ ]:
response = requests.get(URL_OAI_DEFINITION)
apidoc = json.loads(response.text)
ref = apidoc['paths']['/collections/datasets/items']['get']['parameters']
df = pd.json_normalize(ref, max_level = 0)
df[['name','description','x-value']].sort_values(by=['name'])

In [ ]:
jstr = json.dumps(ref, indent=3)
md("```json\n" + jstr + "\n```\n")

```{index} double: OGC API Features ; media types
```
### Search response formats

The following response formats (media types) for `datasets` (granules) are declared in the `/collections/datasets/items` section of the OpenAPI definition.  The media type can be requested via the `Accept` header parameter or the `httpAccept` query parameter.  

In [ ]:
ref = apidoc['paths']['/collections/datasets/items']['get']['responses']['200']['content']
df = pd.json_normalize(ref, max_level = 0)
sorted(ref.keys())

```{index} double: OGC API Features ; parentIdentifier
```

### Search by collection

The `parentIdentifier` parameter is mandatory for granule searches as not all granule metadata is available at a single location.  It allows identifying the Earth Observation (EO) collection to which the granules belong.  In most, but not all cases, the parentIdentifier corresponds to the `fileIdentifier` in the corresponding collection metadata representation in ISO19139(-2) metadata format.  When using the API through the OpenSearch OSDD for granule search, the value of this API parameter is pre-filled automatically through the two-step search mechanism.

**Example: 3.1**  
>  Search granules by (Earth observation) collection (`parentIdentifier`) list. 

In [ ]:
results = w.collection_items(
    collection_id = 'datasets', 
    limit = 5, 
    parentIdentifier=COLLECTION_ID1
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['date', 'title']]

In newer versions of the software the `parentIdentifier` can also be directly used as identifier of the `collection`.  The following request provides the same result.

In [ ]:
results = w.collection_items(
    collection_id = COLLECTION_ID1, 
    limit = 5
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

```{index} double: OGC API Features ; bbox
```

### Search by bounding box

**Example: 3.2**  
>  Search granules by bounding box (`bbox`).  

In [ ]:
results = w.collection_items(
    'datasets', 
    limit=10, 
    bbox=(14.90, 37.700, 14.99, 37.780), # Mount Etna
    parentIdentifier=COLLECTION_ID1
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['date', 'title']]

In [ ]:
jstr = json.dumps(results, indent=3)
md("```json\n" + jstr + "\n```\n")

In [ ]:
display_map(results)

```{index} double: OGC API Features; geometry
```

### Search by geometry

If the parameter is supported for the collection, then the OpenSearch OSDD template identifies one or more profiles of the geo:geometry values that can be used in a granule search request. Possible profiles include searches by point, linestring, multipoint, multilinestring or polygon. In all cases, the geometry value is to be provided in Well-Known Text (WKT) format.

**Example: 3.3**  
>  Search granules by polygon geometry (`geometry`).  

In [ ]:
results = w.collection_items(
    'datasets', 
    limit=3, 
    geometry='POLYGON((14.90 37.700, 14.90 37.780, 14.99 37.780, 14.99 37.700, 14.90 37.700))',
    parentIdentifier=COLLECTION_ID1
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
display_map(results)

**Example: 3.4**  
>  Search granules by point geometry (`geometry`).  

In [ ]:
results = w.collection_items(
    'datasets', 
    limit=3, 
    geometry='POINT(4.3353 51.26866)',  # Antwerp
    parentIdentifier=COLLECTION_ID1
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
display_gdf_plot(results)

In [ ]:
display_map(results)

```{index} double: OGC API Features; datetime
```

### Search by temporal extent

**Example: 3.5**  
>  Search granules by date range (`datetime`).  

In [ ]:
results = w.collection_items(
    'datasets', 
    limit=50, 
    parentIdentifier=COLLECTION_ID1,
    datetime='2019-01-01T00:00:00Z/2019-12-02T00:00:00Z'
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
display_date_distribution(results)

**Example: 3.6**  
>  Search granules by open-ended date range (datetime).  

In [ ]:
results = w.collection_items(
    'datasets', 
    limit=50, 
    parentIdentifier=COLLECTION_ID1,
    datetime='2021-12-01T00:00:00Z/'
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
display_date_distribution(results)

```{index} double: OGC API Features; uid
```

### Search by identifier

The geo:uid combined with the collection identifier eo:parentIdentifier (already prefilled in the OSDD template extracted from the collection search response) allows retrieving granule metadata for a specific granule. Use an identifier extracted from the previous search response.

In [ ]:
# Keep for future use.
granule_id = results['features'][0]['id']
granule_id

**Example: 3.7**  
>  Search granules by identifier (`uid`). 

In [ ]:
results = w.collection_items(
    collection_id = 'datasets', 
    uid = granule_id,
    parentIdentifier = COLLECTION_ID1
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
results

Alternatively, the feature can be accessed directly as shown below.  The `httpAccept` query parameter can be added to request a different representation.

**Example: 3.8**  
>  Access granule by identifier.  

In [ ]:
results = w.collection_item(
    collection_id = 'datasets', 
    identifier = granule_id
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
# results['properties']['title']
# results['properties']
results

```{index} double: OGC API Features; acquisition parameters
```
### Search by acquisition parameters


```{index} double: OGC API Features; illuminationElevationAngle
```
```{index} double: OGC API Features; range of values
```
**Example: 3.9**  
>  Search granules by illumination angles `illuminationElevationAngle`, `illuminationAzimuthAngle`.

The example shows how a set of values can be provided using square brackets: `[`value-1,value-2`]`.

The `illuminationElevationAngle` and `illuminationAzimuthAngle` search parameters allow filtering results by illumination angles. An interval specifying minimum and maximum allowed values is to be provided, e.g. `[48,50]`. Only providing the minimum or maximum value can be done by using an open interval, e.g. `[48` or `50]`.

Other acquisition parameters can be advertised as searchable in the collection OSDD, depending on the sensor type e.g.:

- orbitNumber
- orbitDirection
- frame
- track
etc.

In [ ]:
results = w.collection_items(
    limit = 3,
    collection_id = 'datasets', 
    parentIdentifier = COLLECTION_ID3_CLOUDS,
    illuminationElevationAngle = '[30,40]'  
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
df = pd.json_normalize(results['features'][0]['properties']['acquisitionInformation'])
df.transpose()

```{index} double: OGC API Features; orbitDirection
```
**Example: 3.10**  
>  Search granules by `orbitDirection`.

In [ ]:
results = w.collection_items(
    limit = 3,
    collection_id = 'datasets', 
    parentIdentifier = COLLECTION_ID2,
    orbitDirection = 'DESCENDING'  
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
df = pd.json_normalize(results['features'][0]['properties']['acquisitionInformation'])
df.transpose()

```{index} double: OGC API Features; orbitNumber
```
```{index} double: OGC API Features; set of values
```
**Example: 3.11**  
>  Search granules by `orbitNumber`.

The example shows how a set of values can be provided using curly brackets: `{`value-1,value-2,...,value-n`}`

In [ ]:
results = w.collection_items(
    limit = 3,
    collection_id = 'datasets', 
    parentIdentifier = COLLECTION_ID2_CLOUDS,
    orbitNumber = '{1237, 1248}'  # 1237 or 1248
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
df = pd.json_normalize(results['features'][0]['properties']['acquisitionInformation'])
df.transpose()

## Advanced topics

### Sorting results

```{index} double: OGC API Features ; sortKeys
```
```{index} double: searchRetrieve ; sortKeys
```

Sorting of search results is available for collection, services and granule searches.   The supported search criteria can be found in the corresponding OpenSearch OSDD document.  The `sortKeys` query parameter is defined in the SRU specification [[RD07]](#RD07).


**Example: 4.1**  
>  Collection search results can be sorted according to various criteria with `sortKeys` [[RD07]](#RD07), in descending or ascending order which can be discovered in the OSDD. The example sorts collections in descending chronological order according to the {eo:modificationDate} value. 

In [ ]:
results = w.collection_items(
    'series', 
    limit=10, 
    # sortKeys='title,dc,1',
    sortKeys='modificationDate,eo,0',
    organisationName = 'ESA/ESRIN' 
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['updated','title']]

**Example: 4.2**  
>  Granule search results can be sorted according to various criteria with `sortKeys` [RD07], in descending or ascending order which can be discovered in the OSDD. The example sorts granules in ascending chronological order according to the {time:start} value. 

In [ ]:
results = w.collection_items(
    'datasets', 
    limit=10, 
    parentIdentifier=COLLECTION_ID1,
    sortKeys='start,time,1'
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
gdf = gpd.GeoDataFrame.from_features(results)
gdf[['date','title']]

### Faceted search

```{index} double: OGC API Features ; faceted search
```
```{index} double: OGC API Features ; facetLimit
```
```{index} double: searchRetrieve ; facetLimit
```

Faceted search results are available for collection, service and granule searches in both GeoJSON and Atom response formats.  The GeoJSON encoding is defined in [[RD35]](#RD35) at https://docs.ogc.org/per/19-020r1.html#_facetedresults and consistent with the OASIS SRU encoding available for Atom response format defined in [[RD07]](#RD07).

The server can supply faceted results for a query: i.e. an analysis of how the search results are distributed over various categories (or “facets”). For example, the analysis may reveal how the results are distributed by organization. The client might then refine the query to one particular organization among those listed.

By default, the search response contains faceted search results under the element facetedResults. In the current implementation, the faceted search information groups the results by a number of predefined facets which may include platform, instrument and organisation etc. The client can also specify which facets it would like to receive as part of the request with the `facetLimit` search parameter.

The search response does not include the list of all values for the facet. The list is truncated and provides the values with the largest count. Future versions of the software may allow paging through all values, which would require support for the sru:facetStart search parameter.

The faceted Results are consistent with the OASIS searchRetrieve facetedResults XML schema available at http://docs.oasis-open.org/search-ws/searchRetrieve/v1.0/os/schemas/facetedResults.xsd defined in [[RD07]](#RD07) and the GeoJSON encoding defined in [[RD35]](#RD35).

The `facetLimit` query parameter is defined in the SRU specification [[RD07]](#RD07).

```{index} double: OGC API Features ; Turtle
```
```{index} double: OGC API Features ; Atom
```
```{index} double: OGC API Features ; JSON-LD
```
```{index} double: OGC API Features ; RDF/XML
```
```{index} double: OGC API Features ; httpAccept
```
```{index} double: OGC API Features ; Accept
```
```{index} double: searchRetrieve ; httpAccept
```

(all-response-formats)=
### Additional response formats

Different representations are available for search results (container), and individual items representing a Collection, Service or Granule. They are declared in the OpenAPI definition document (`/api`).  Content negotiation can be used via the `Accept` header parameter or `httpAccept` query parameter.
Available formats are listed in the following subsections.

All the search criteria described previously can be combined with any of the available representations.

In [ ]:
md(f"The OpenAPI definition describing the OGC API Features interface is available at `{URL_OAI_DEFINITION}`.")

#### Search results

The various media types available for search results are listed in the OpenAPI definition.  For each of the available media types, an example is provided in the current section.

In [ ]:
ref = apidoc['paths']['/collections/datasets/items']['get']['responses']['200']['content']
df = pd.json_normalize(ref, max_level = 0)
sorted(ref.keys())

In [ ]:
html_str = create_example_table_as_html(
    mediatypes = sorted(ref.keys()),
    chapter = 5)
HTML(html_str)

In [ ]:
md_str = create_examples_as_md(
    mediatypes = sorted(ref.keys()),
    chapter = 5, 
    resource = URL_LANDING_PAGE + 'collections/series/items?limit=0',
    resource_type = "search response" )
md(md_str)

#### Item (Collection)

The various media types available for representing a collection are listed in the OpenAPI definition.  For each of the available media types, an example is provided in the current section.

In [ ]:
ref = apidoc['paths']['/collections/series/items/{seriesId}']['get']['responses']['200']['content']
df = pd.json_normalize(ref, max_level = 0)
sorted(ref.keys())

In [ ]:
# Generate a markdown table of content with links to all individual formats 
# as they cannot be included in the TOC or index.

# Table replaced by list as markdown table is not corectly converted...

text="| Example | Media type | Reference |" + '\n' \
    + "| --- | --- | --- | " + '\n'

digit1 = 6
digit2 = 1
for f in sorted(ref.keys()):
    
    f2 = f
    # f2 = f.replace(";", ";<br>")  # works in VS-Code, not in JupyterBook
    # f2 = f2.replace('"', '\\"')
    #       + '| ' + 'Example '+str(digit1)+ '.' + str(digit2)  \
     
    text = text \
        + '| ' + '[Example '+str(digit1)+ '.' + str(digit2)+'](#example_' + str(digit1) + "_" + str(digit2) + ') ' \
        + '| ' + f2  + ' | ' + "tbd" + ' |' + '\n'
    digit2 = digit2+1
    
md(text)

In [ ]:
html_str = create_example_table_as_html(
    mediatypes = sorted(ref.keys()),
    chapter = 6)
HTML(html_str)

In [ ]:
series_id

In [ ]:
md_str = create_examples_as_md(
    mediatypes = sorted(ref.keys()),
    chapter = 6, 
    resource = URL_LANDING_PAGE + 'collections/series/items/' + series_id,
    resource_type = "series" )
md(md_str)

#### Item (Service)

The various media types available for representing a service (or application) are listed in the OpenAPI definition.  For each of the available media types, an example is provided in the current section.

In [ ]:
ref = apidoc['paths']['/collections/services/items/{serviceId}']['get']['responses']['200']['content']
df = pd.json_normalize(ref, max_level = 0)
sorted(ref.keys())

In [ ]:
html_str = create_example_table_as_html(
    mediatypes = sorted(ref.keys()),
    chapter = 7)
HTML(html_str)

In [ ]:
service_id = 'rasdaman'

In [ ]:
md_str = create_examples_as_md(
    mediatypes = sorted(ref.keys()),
    chapter = 7, 
    resource = URL_LANDING_PAGE + 'collections/services/items/' + service_id,
    resource_type = "service" )
md(md_str)

#### Item (Granule)

The various media types available for representing a granule are listed in the OpenAPI definition.  For each of the available media types, an example is provided in the current section.

In [ ]:
ref = apidoc['paths']['/collections/datasets/items/{datasetId}']['get']['responses']['200']['content']
df = pd.json_normalize(ref, max_level = 0)
sorted(ref.keys())

In [ ]:
html_str = create_example_table_as_html(
    mediatypes = sorted(ref.keys()),
    chapter = 8)
HTML(html_str)

In [ ]:
granule_id1 = 'PR1_OPER_CHR_MO2_1P_20161003T154700_N36-095_W006-036_0001'

In [ ]:
md_str = create_examples_as_md(
    mediatypes = sorted(ref.keys()),
    chapter = 8, 
    resource = URL_LANDING_PAGE + 'collections/datasets/items/' + granule_id1,
    resource_type = "granule" )
md(md_str)

### Linked Data

The available representations include the RDF serialisations [RDF/XML](#RD37), [JSON-LD](#RD36) and [Turtle](#RD38) which allow representing the different resources as linked data.  Representations according to [schema.org](#RD40) and [GeoDCAT-AP](#RD39) are supported.

**Example: 9.1**  
>  Represent collection as linked data with Turtle and [GeoDCAT-AP](#RD39).

In [ ]:
resource = URL_LANDING_PAGE + 'collections/series/items/' + series_id
f = 'text/turtle;profile="http://data.europa.eu/930/"'
url = resource +'?httpAccept='+urllib.parse.quote(f)

In [ ]:
curl_str = curl_command(url)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(url)
# response.text
md("```\n" + response.text + "\n```\n")

In [ ]:
# display Turtle as graph
from matplotlib.pyplot import figure
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_graph
import networkx as nx

figure(figsize=(30, 40), dpi=80)
rg = rdflib.Graph()

# TBD: Remove $$ signs from input to avoid bug in JSON-LD to Turtle conversion.
rg.parse(data=response.text.replace("$$", "" ), format='ttl', encoding='utf-8')
G = rdflib_to_networkx_graph(rg)
plt.plot()
nx.draw(G, with_labels=True)

**Example: 9.2**  
>  Represent collection as linked data with Turtle and [Schema.org](#RD40).

In [ ]:
resource = URL_LANDING_PAGE + 'collections/series/items/' + series_id
f = 'text/turtle;profile="https://schema.org"'
url = resource +'?httpAccept='+urllib.parse.quote(f)

In [ ]:
curl_str = curl_command(url)
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
response = requests.get(url)
# response.text
md("```\n" + response.text + "\n```\n")

In [ ]:
# display Turtle as graph
from matplotlib.pyplot import figure
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_graph
import networkx as nx

figure(figsize=(30, 40), dpi=80)
rg = rdflib.Graph()
# TBD: Remove $$ signs from input to avoid bug in JSON-LD to Turtle conversion.
rg.parse(data=response.text.replace("$$", "" ), format='ttl', encoding='utf-8')
# rg.parse(data=response.text, format='ttl', encoding='utf-8')
G = rdflib_to_networkx_graph(rg)
plt.plot()
nx.draw(G, with_labels=True)

### Additional search parameters

Additional search parameters beyond the OGC API Features search parameters can be used to filter results (See also https://docs.opengeospatial.org/is/17-069r3/17-069r3.html#_parameters_for_filtering_on_feature_properties).  They are defined in the OpenAPI definition as additional HTTP query parameters equivalent to the available OpenSearch parameters. 

Not all search parameters apply to all collections.  The available parameters for each collection are advertised in the corresponding `/collections/{collection-id}/queryables` response.  They may be used as additional HTTP query parameters or with the `filter` parameter, as the interface supports the corresponding conformance classes:

- `http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/queryables` from OGC API Features Part 3 [[RD21]](#RD21) indicates that collection-specific search parameters are advertised at `/collections/{collection-id}/queryables`.
- `http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/queryables-query-parameters` from OGC API Features Part 3 [[RD21]](#RD21) indicates that queryables advertised at `/collections/{collection-id}/queryables` can be used as HTTP query parameter.
- `http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/filter` from OGC API Features Part 3 [[RD21]](#RD21) indicates that queryables advertised at `/collections/{collection-id}/queryables` can be used in a CQL2 filter expression.

The same search parameter listed in the /queryables response can thus be used as HTTP query parameter or inside a filter expression.  When used as HTTP query parameter, only the equality "=" can be used and the OpenSearch conventions apply, e.g. illuminationElevationAngle=[10,55].  When the same parameter is used inside a filter expression, a CQL expression is to be used with the comparision predicates "<=" and ">=".

#### Conformance class Queryables

**Example: 10.1**  
>  Collections advertise the list of search parameters they support in a Queryables object in JSON Schema format.

In [ ]:
URL = URL_LANDING_PAGE + "collections/" + COLLECTION_ID1 + "/queryables"
URL

In [ ]:
response = requests.get(URL)   
data = json.loads(response.text)    
df = pd.DataFrame(data['properties'].items(),columns=['key','value'])
df['type'] = df.apply(lambda row : row[1]['type'], axis = 1)
df['format'] = df.apply(lambda row : row[1]['format'] if 'format' in row[1] else '-' , axis = 1)
df.drop('value',axis=1).sort_values(by=['key'])

In [ ]:
jstr = json.dumps(data, indent=3)
md("```json\n" + jstr + "\n```\n")

**Example: 10.2**  
>  List of search parameters available for collection search returned in a Queryables object in JSON Schema format.

In [ ]:
URL = URL_LANDING_PAGE + "collections/" + "series" + "/queryables"
URL

In [ ]:
response = requests.get(URL)   
data = json.loads(response.text)    
df = pd.DataFrame(data['properties'].items(),columns=['key','value'])
df['type'] = df.apply(lambda row : row[1]['type'], axis = 1)
df['format'] = df.apply(lambda row : row[1]['format'] if 'format' in row[1] else '-' , axis = 1)
df.drop('value',axis=1).sort_values(by=['key'])

In [ ]:
jstr = json.dumps(data, indent=3)
md("```json\n" + jstr + "\n```\n")

```{index} double: OGC API Features ; cql-text
```

(conformance-class-filter)=
#### Conformance class Filter

The interface supports the `filter` parameter and filter expressions expressed with the Text encoding `cql-text` of the Basic Common Query Language (Basic CQL2-Text) [[RD22]](#RD22).

```{index} double: CQL2-Text ; logical operators
```

**Example: 11.1**  
>  CQL Filter with logical operators (and, or).

In [ ]:
results = w.collection_items(
    collection_id = 'series', 
    limit = 10, 
    filter = "platform = 'Envisat' and ( instrument = 'MERIS' or instrument = 'ASAR' )  and organisationName = 'ESA/ESRIN'"
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")

In [ ]:
pTable = PrettyTable()
pTable.align = "l"
pTable.field_names = ["Id", "Platforms / Instruments"]

for item in results['features']:
    id = item['id']
    platform = ''
    first = True
    
    for acq in item['properties']['acquisitionInformation']:                
        if first:
            first = False
        else:
            platform += ', '
        if "platform" in acq:
            platform += acq['platform']['platformShortName']
        if "instrument" in acq:
            platform += "/" + acq['instrument']['instrumentShortName'] 
    
    pTable.add_row([id,platform])

print(pTable)

**Example: 11.2**  
>  CQL Filter with logical operators (and, not).

In [ ]:
results = w.collection_items(
    collection_id = 'series', 
    limit = 10, 
    filter = "(platform = 'Envisat') and ( NOT (instrument = 'MERIS') )  and (organisationName = 'ESA/ESRIN')"
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")
# assert results['numberMatched'] > 0

In [ ]:
pTable = PrettyTable()
pTable.align = "l"
pTable.field_names = ["Id", "Platforms / Instruments"]

for item in results['features']:
    id = item['id']
    platform = ''
    first = True
    
    for acq in item['properties']['acquisitionInformation']:                
        if first:
            first = False
        else:
            platform += ', '
        if "platform" in acq:
            platform += acq['platform']['platformShortName']
        if "instrument" in acq:
            platform += "/" + acq['instrument']['instrumentShortName'] 
    
    pTable.add_row([id,platform])

print(pTable)

```{index} double: CQL2-Text ; IS NULL predicate
```
```{index} double: CQL2-Text ; TIMESTAMP literal
```

**Example: 11.3**  
>  CQL filter with IS NULL predicate and timestamp literal.

In [ ]:
results = w.collection_items(
    collection_id = 'series', 
    limit = 10, 
    filter = "otherConstraint is null and modificationDate > TIMESTAMP('2019-01-01T20:17:40Z') and organisationName = 'ESA/ESRIN'"
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")
assert results['numberMatched'] > 0

In [ ]:
pTable = PrettyTable()
pTable.align = "l"
pTable.field_names = ["Id", "Modification date"]

for item in results['features']:
    pTable.add_row([item['id'],item['properties']['updated']])

print(pTable)  

```{index} double: CQL2-Text ; comparison operators
```

**Example: 11.4**  
>  CQL filter with comparison operators.  Search granules by cloudCover >= 10 and cloudCover < 20 

In [ ]:
results = w.collection_items(
    collection_id = 'IKONOS.ESA.archive',  
    limit = 20, 
    filter = "cloudCover >= 10 and cloudCover < 20"
)

In [ ]:
curl_str = curl_command(w.url_with_parameters())
md("```shell\n" + curl_str + "\n```\n")

In [ ]:
print(f"{results['numberMatched']} items found.")
assert results['numberMatched'] > 0

In [ ]:
pTable = PrettyTable()
pTable.align = "l"
pTable.field_names = ["Id", "Cloud cover"]

for item in results['features']:
    pTable.add_row([item['id'],str(item['properties']['productInformation']['cloudCover'])])

print(pTable) 

## Further Reading

| **ID**  | **Title** | 
| -------- | --------- | 
| `RD07` <a name="RD07"></a> | [OASIS searchRetrieve: Part 3. APD Binding for SRU 2.0 Version 1.0](http://docs.oasis-open.org/search-ws/searchRetrieve/v1.0/os/part3-sru2.0/searchRetrieve-v1.0-os-part3-sru2.0.html) |
| `RD19` <a name="RD19"></a> | [ODC STAC - Plot STAC Items on a map ](https://odc-stac.readthedocs.io/en/latest/notebooks/stac-load-e84-aws.html#Plot-STAC-Items-on-a-Map) | 
| `RD20` <a name="RD20"></a> | [OGC17-069r3, OGC API - Features - Part 1: Core](https://docs.opengeospatial.org/is/17-069r3/17-069r3.html) | 
| `RD21` <a name="RD21"></a> | [OGC17-079r1, OGC API - Features - Part 3: Filtering](https://docs.opengeospatial.org/DRAFTS/19-079r1.html)  | 
| `RD22` <a name="RD22"></a> | [OGC21-065, Common Query Language (CQL2)](https://docs.ogc.org/DRAFTS/21-065.html)  | 
| `RD23` <a name="RD23"></a> | [RFC 7946 - The GeoJSON Format](https://datatracker.ietf.org/doc/html/rfc7946) | 
| `RD24` <a name="RD24"></a>| [JSON Schema: A Media Type for Describing JSON Documents, draft-handrews-json-schema-02](https://datatracker.ietf.org/doc/html/draft-handrews-json-schema-02) |
| `RD25` <a name="RD25"></a>| [STAC API - Collection Search](https://github.com/stac-api-extensions/collection-search) |
| `RD26` <a name="RD26"></a>| [STAC API - Filter Extension](https://github.com/stac-api-extensions/filter) |
| `RD31` <a name="RD31"></a>| [OWSLib - Usage](https://geopython.github.io/OWSLib/usage.html#ogc-api) |
| `RD32` <a name="RD32"></a> | [OGC17-003r2, OGC EO Dataset Metadata GeoJSON(-LD) Encoding Standard](https://docs.opengeospatial.org/is/17-003r2/17-003r2.html)  | 
| `RD33` <a name="RD33"></a> | [OGC17-047r1, OGC OpenSearch-EO GeoJSON(-LD) Response Encoding Standard](https://docs.ogc.org/is/17-047r1/17-047r1.html)  | 
| `RD34` <a name="RD34"></a> | [OGC17-084r1, EO Collection GeoJSON(-LD) Encoding Best Practice](https://docs.ogc.org/bp/17-084r1/17-084r1.html)  |
| `RD35` <a name="RD35"></a> | [OGC19-020r1, OGC Testbed-15: Catalogue and Discovery Engineering Report](https://docs.ogc.org/per/19-020r1.html)  |
| `RD36` <a name="RD36"></a> | [JSON-LD 1.1, A JSON-based Serialization for Linked Data, W3C Recommendation 16 July 2020](https://www.w3.org/TR/json-ld11/)  |
| `RD37` <a name="RD37"></a> | [RDF 1.1 XML Syntax, W3C Recommendation 25 February 2014](http://www.w3.org/TR/rdfsyntax-grammar/)  |
| `RD38` <a name="RD38"></a> | [RDF 1.1 Turtle, Terse RDF Triple Language, W3C Recommendation 25 February 2014](http://www.w3.org/TR/turtle/)  |
| `RD39` <a name="RD39"></a> | [GeoDCAT-AP Version 2.0.0, European Commission](https://semiceu.github.io/GeoDCAT-AP/releases/2.0.0/)  |
| `RD40` <a name="RD40"></a> | [Schema.org](https://schema.org/)  |




